## 进程和线程
对于操作系统来说，一个任务就是一个进程，有的进程存在至少一个线程，线程之间快速切换让我们有一种线程同步运行的错觉  
python支持多进程和多线程，但是最好优先选择多线程执行多任务的功能。　　

## 多进程
详见Chart1.md

## 多进程
我们通常使用threading该模块实现我们的多线程操作　　


In [2]:
import time , threading

def loop():
    print("thread %s is running ..." % threading.current_thread().name)
    n = 0
    while n<5:
        n += 1
        print("thread %s >>> %s " % (threading.current_thread().name , n))
        time.sleep(1)
    print("thread %s ended ..." % threading.current_thread().name)

print("%s is running ..." % threading.current_thread().name)
t = threading.Thread(target = loop , name = 'loopthread')
t.start()
t.join()
print("thread %s ended..." % threading.current_thread().name)

MainThread is running ...
thread loopthread is running ...
thread loopthread >>> 1 
thread loopthread >>> 2 
thread loopthread >>> 3 
thread loopthread >>> 4 
thread loopthread >>> 5 
thread loopthread ended ...
thread MainThread ended...


任何一个进程都会默认的启动一个线程，该线程是主线程，主线程可以分成其他的多个线程
* threading.current_thread()返回当前线程的实例

### 线程锁
在多进程中，每个进程的资源都是独立的不互相影响，但是线程不一样，线程之间是资源共享的，这就带来一个问题，如果多个线程读写同一个数据，那么很有可能会出现意料之外的结果  
如下的情况既有可能的原因就是因为语句  
balance += n / balance -= n　　
这两个语句中都是寄到一个局部变量，**不同的线程有用自己的局部变量**，会出现读取脏数据的丢失更新的情况　　
这时候，我们就必须给我们的工作函数加上一个锁，同一时刻只有一个线程可以持有该锁，有该锁的线程才有资格修改变量

In [8]:
import time , threading

balance = 0

def change(n):
    global balance
    balance += n
    balance -= n

def runthread(n):
    for i in range(500000):
        change(n)

t1 = threading.Thread(target = runthread , args=(5,))
t2 = threading.Thread(target = runthread , args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

3


In [12]:
# 有了锁，并发程度降低，但是线程安全
import time , threading

lock = threading.Lock()    # Add the lock

balance = 0

def change(n):
    global balance
    balance += n
    balance -= n

def runthread(n):
    lock.acquire()    # 获取锁
    try:
        for i in range(1000000):
            change(n)
    finally:
        lock.release()    # 释放锁

t1 = threading.Thread(target = runthread , args=(5,))
t2 = threading.Thread(target = runthread , args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


### threadlocal
刚才说过了，每个线程都自己的局部变量，避免影响其他的线程，**全局变量修改必须加锁，局部不用**   
但是我们的线程内的局部变量传参太麻烦了，有两种解决方式   
* 全局字典   
  全局字典以threading.current_thread()作为键，以线程局部变量作为值，以便之后都可以使用，但是如果有多个局部变量呢?使用threading.local  
* threading.local()对象：  
  **threading.local()对象**可以通过'.'作用域添加多个局部变量和访问，并且不同的进程之间是互不干扰的  